# Introduction

We  will try to retrain a pretrained inception_v3 model on the images. We will apply augmentations to the images and we will rescale to images to 300x300 pixels.
The augmentations will be performed using Keras preprocessing libraries and the Data Generator objects.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow  as  tf
import tensorflow.keras  as  keras

# Load the data

* Load the train.csv file that contains the image labels
* Load the labels map file

In [ ]:
import  json

df=pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
df.head()
df["label"]=df["label"].astype(str)
train_ratio=0.9
df_train=df[:int(train_ratio*len(df))]
df_valid=df[int(train_ratio*len(df)):]

f=open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json")
labels=json.loads(f.read())

# Labels Distribution

* Are the data ballanced?

In [ ]:
group=df_train.groupby(["label"]).agg("count")
from matplotlib import  pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.pie(group.values,labels=group.index)
plt.legend([v for k,v in labels.items()])
plt.show()

# Plot Images

* Lets plot some images  in order to see how the look like.

In [ ]:
import numpy as np
from PIL import Image


def plot_images(df,label=0,num_images=10,title=""):
    fig=figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    df=df[df["label"]==label]
    i=0
    size = np.sqrt(num_images)
    for index,row in df.iterrows():
        i=i+1
        if i>num_images:
            break
        plt.subplot(size, size,i)
        img = Image.open('../input/cassava-leaf-disease-classification/train_images/'+row["image_id"])
        plt.imshow(img)
    fig.suptitle(title, fontsize=20)
    plt.show()

In [ ]:
k="0"
v=labels[k]
plot_images(df_train,label=k,num_images=16,title=v)

In [ ]:
k="1"
v=labels[k]
plot_images(df_train,label=k,num_images=16,title=v)

In [ ]:
k="2"
v=labels[k]
plot_images(df_train,label=k,num_images=16,title=v)

In [ ]:
k="3"
v=labels[k]
plot_images(df_train,label=k,num_images=16,title=v)

In [ ]:
k="4"
v=labels[k]
plot_images(df_train,label=k,num_images=16,title=v)

# Directories for the Training

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size=300
batch_size=32

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=[1.0,2.0],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_dataframe(
        dataframe = df_train,
        directory='../input/cassava-leaf-disease-classification/train_images',
        x_col = 'image_id',
        y_col = 'label',     
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='sparse') 
valid_datagen = ImageDataGenerator(
      rescale=1./255)
valid_generator = valid_datagen.flow_from_dataframe(
        dataframe = df_valid,
        directory='../input/cassava-leaf-disease-classification/train_images',
        x_col = 'image_id',
        y_col = 'label',     
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='sparse') 

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import EfficientNetB3
pre_trained_model = InceptionV3(input_shape = (img_size, img_size, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights('../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

input_shape=(img_size,img_size,3)
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed10')
last_output = last_layer.output
from tensorflow.keras.optimizers import RMSprop

# x = layers.Flatten()(last_output)
x = layers.GlobalAveragePooling2D()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x) 
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (5, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 


callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5),
             EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=10,
      verbose=1,
      validation_data=valid_generator,
      validation_steps=8,callbacks=callbacks)

In [ ]:
history_frame = pd.DataFrame(history.history)

In [ ]:
history_frame

In [ ]:
from keras.preprocessing import image
import os


image_ids=[]
labels=[]
imgs=[]
for file in os.listdir('../input/cassava-leaf-disease-classification/test_images/'):
    img = image.load_img('../input/cassava-leaf-disease-classification/test_images/'+file, target_size=(img_size, img_size))
    x = image.img_to_array(img)
    x=x/255.0
    imgs.append(x)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    image_ids.append(file)
imgs=np.array(imgs)
classes = model.predict(imgs)
labels=np.argmax(classes,axis=1)
submission_df=pd.DataFrame()
submission_df["image_id"]=image_ids
submission_df["label"]=labels

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)